# RECOMMENDATION SYSTEM 1

**MODEL 1 -- COLLABORATIVE MODEL**

In [13]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs
# Ratings data.
ratings = tfds.load('movielens/100k-ratings', split="train")
# Features of all the available movies.
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features.
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"]
})
movies = movies.map(lambda x: x["movie_title"])
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["user_id"])
    movie_embeddings = self.movie_model(features["movie_title"])

    return self.task(user_embeddings, movie_embeddings)
   # Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/3
25/25 [==============================] - 18s 667ms/step - factorized_top_k/top_1_categorical_accuracy: 7.0000e-05 - factorized_top_k/top_5_categorical_accuracy: 0.0015 - factorized_top_k/top_10_categorical_accuracy: 0.0047 - factorized_top_k/top_50_categorical_accuracy: 0.0431 - factorized_top_k/top_100_categorical_accuracy: 0.0977 - loss: 33096.4720 - regularization_loss: 0.0000e+00 - total_loss: 33096.4720
Epoch 2/3
25/25 [==============================] - 15s 604ms/step - factorized_top_k/top_1_categorical_accuracy: 1.6000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0051 - factorized_top_k/top_10_categorical_accuracy: 0.0138 - factorized_top_k/top_50_categorical_accuracy: 0.1046 - factorized_top_k/top_100_categorical_accuracy: 0.2094 - loss: 31007.5086 - regularization_loss: 0.0000e+00 - total_loss: 31007.5086
Epoch 3/3
25/25 [==============================] - 15s 603ms/step - factorized_top_k/top_1_categorical_accuracy: 4.2000e-04 - factorized_top_k/top_5_categor

## RECOMMENDATION SYSTEM 2

**Model 1: Collaborative Filtering using Matrix Factorization**

In [14]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Load data
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})
movies = movies.map(lambda x: x["movie_title"])

# Create and adapt the vocabulary layers
user_ids_vocabulary = tf.keras.layers.StringLookup()
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup()
movie_titles_vocabulary.adapt(movies)

# Define the model
class MatrixFactorizationModel(tfrs.Model):
    def __init__(self, user_model, movie_model, rating_model, task):
        super().__init__()
        self.user_model = user_model
        self.movie_model = movie_model
        self.rating_model = rating_model
        self.task = task

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])
        ratings = features["user_rating"]

        predicted_ratings = self.rating_model(tf.concat([user_embeddings, movie_embeddings], axis=1))
        return self.task(ratings, predicted_ratings)

# User and movie models
embedding_dimension = 32
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), embedding_dimension)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), embedding_dimension)
])

# Rating model
rating_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=256, activation="relu"),
    tf.keras.layers.Dense(units=64, activation="relu"),
    tf.keras.layers.Dense(units=1)
])

# Task and model
task = tfrs.tasks.Ranking(loss=tf.keras.losses.MeanSquaredError())
model = MatrixFactorizationModel(user_model, movie_model, rating_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Training
model.fit(ratings.batch(4096), epochs=3)

# Evaluation
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(movies.batch(100).map(lambda title: (title, model.movie_model(title))))

# Recommendations
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")



Epoch 1/3
25/25 [==============================] - 4s 143ms/step - loss: 2.4793 - regularization_loss: 0.0000e+00 - total_loss: 2.4793
Epoch 2/3
25/25 [==============================] - 1s 56ms/step - loss: 1.1939 - regularization_loss: 0.0000e+00 - total_loss: 1.1939
Epoch 3/3
25/25 [==============================] - 1s 58ms/step - loss: 1.0737 - regularization_loss: 0.0000e+00 - total_loss: 1.0737
Top 3 recommendations for user 42: [b'Basquiat (1996)' b'Touch of Evil (1958)'
 b'Great Day in Harlem, A (1994)']


# RECOMMENDATION SYSTEM 3

**Model 2: Hybrid Model (Collaborative Filtering + Content-Based Filtering)**

In [15]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

# Load data
ratings = tfds.load('movielens/100k-ratings', split="train")
movies = tfds.load('movielens/100k-movies', split="train")

# Select the basic features
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
    "user_rating": x["user_rating"]
})
movies = movies.map(lambda x: {
    "movie_title": x["movie_title"]
})

# Define the model
class HybridModel(tfrs.Model):
    def __init__(self, user_model, movie_model, rating_model, task):
        super().__init__()
        self.user_model = user_model
        self.movie_model = movie_model
        self.rating_model = rating_model
        self.task = task

    def compute_loss(self, features, training=False):
        user_embeddings = self.user_model(features["user_id"])
        movie_embeddings = self.movie_model(features["movie_title"])
        ratings = features["user_rating"]

        predicted_ratings = self.rating_model(tf.concat([user_embeddings, movie_embeddings], axis=1))
        return self.task(ratings, predicted_ratings)

# User and movie models
embedding_dimension = 32

# Prepare StringLookup layers to adapt to the actual vocabulary
user_ids_vocabulary = tf.keras.layers.StringLookup()
movie_titles_vocabulary = tf.keras.layers.StringLookup()

# Adapt the vocabularies to the dataset
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))
movie_titles_vocabulary.adapt(ratings.map(lambda x: x["movie_title"]))

user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(input_dim=user_ids_vocabulary.vocab_size(), output_dim=embedding_dimension)
])

movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(input_dim=movie_titles_vocabulary.vocab_size(), output_dim=embedding_dimension)
])

# Rating model
rating_model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=256, activation="relu"),
    tf.keras.layers.Dense(units=64, activation="relu"),
    tf.keras.layers.Dense(units=1)
])

# Task and model
task = tfrs.tasks.Ranking(loss=tf.keras.losses.MeanSquaredError())
model = HybridModel(user_model, movie_model, rating_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

# Training
model.fit(ratings.batch(4096), epochs=3)

# Evaluation
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda x: (x["movie_title"], model.movie_model(x["movie_title"])))
)

# Recommendations
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")


Epoch 1/3
25/25 [==============================] - 4s 113ms/step - loss: 2.4940 - regularization_loss: 0.0000e+00 - total_loss: 2.4940
Epoch 2/3
25/25 [==============================] - 1s 54ms/step - loss: 1.2140 - regularization_loss: 0.0000e+00 - total_loss: 1.2140
Epoch 3/3
25/25 [==============================] - 1s 51ms/step - loss: 1.1106 - regularization_loss: 0.0000e+00 - total_loss: 1.1106
Top 3 recommendations for user 42: [b'Citizen Kane (1941)' b'Four Days in September (1997)'
 b'Bonheur, Le (1965)']
